In [4]:
from datasets import load_dataset
from transformers import pipeline
import transformers as trf

import re  #preprocessing
import pandas as pd  #data handling
from time import time  #time-to-run
from collections import defaultdict  #word frequency

import spacy  #preprocessing/tokenization

import logging  #gensim logging
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

from tqdm import tqdm

import matplotlib.pyplot as plt

from sklearn.feature_extraction import text as sktext

import scipy as sp
import numpy as np

import sklearn as sk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import gensim
from gensim.models import Word2Vec, Doc2Vec
from joblib import dump, load

In [2]:
%%time
doc2vec_model = Doc2Vec.load('doc2vec.model')
word2vec_model = Doc2Vec.load('word2vec2.model')

INFO - 15:46:08: loading Doc2Vec object from doc2vec.model
INFO - 15:46:08: loading dv recursively from doc2vec.model.dv.* with mmap=None
INFO - 15:46:08: loading vectors from doc2vec.model.dv.vectors.npy with mmap=None
INFO - 15:46:08: loading wv recursively from doc2vec.model.wv.* with mmap=None
INFO - 15:46:08: loading vectors from doc2vec.model.wv.vectors.npy with mmap=None
INFO - 15:46:09: loading syn1neg from doc2vec.model.syn1neg.npy with mmap=None
INFO - 15:46:09: setting ignored attribute cum_table to None
INFO - 15:46:11: Doc2Vec lifecycle event {'fname': 'doc2vec.model', 'datetime': '2023-04-04T15:46:11.733902', 'gensim': '4.3.0', 'python': '3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:42:03) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'loaded'}


CPU times: total: 2.83 s
Wall time: 3.28 s


In [3]:
%%time
word2vec_model = Doc2Vec.load('word2vec2.model')

INFO - 15:46:18: loading Doc2Vec object from word2vec2.model
INFO - 15:46:19: loading wv recursively from word2vec2.model.wv.* with mmap=None
INFO - 15:46:19: loading vectors from word2vec2.model.wv.vectors.npy with mmap=None
INFO - 15:46:19: loading syn1neg from word2vec2.model.syn1neg.npy with mmap=None
INFO - 15:46:20: setting ignored attribute cum_table to None
INFO - 15:46:20: Word2Vec lifecycle event {'fname': 'word2vec2.model', 'datetime': '2023-04-04T15:46:20.955268', 'gensim': '4.3.0', 'python': '3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:42:03) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'loaded'}


CPU times: total: 1.78 s
Wall time: 2 s


In [6]:
%time

bow1 = load('bow1.jbl')
bow2 = load('bow2.jbl')
tfidf1 = load('tfidf1.jbl')
tfidf2 = load('tfidf2.jbl')